<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Models" data-toc-modified-id="Models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#Predict" data-toc-modified-id="Predict-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Predict</a></span></li><li><span><a href="#Visualize" data-toc-modified-id="Visualize-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Visualize</a></span></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Models" data-toc-modified-id="Models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#Predict" data-toc-modified-id="Predict-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Predict</a></span></li><li><span><a href="#Visualize" data-toc-modified-id="Visualize-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Visualize</a></span></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

In [ ]:
#default_exp models

In [ ]:
#export
from mantisshrimp.imports import *
from mantisshrimp.core import *
from mantisshrimp.data.all import *

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Models" data-toc-modified-id="Models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#Predict" data-toc-modified-id="Predict-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Predict</a></span></li><li><span><a href="#Visualize" data-toc-modified-id="Visualize-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Visualize</a></span></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

# Models
> Models for object detection

In [ ]:
#export
class RCNN(LightningModule):
    def __init__(self, model_splitter=None, metrics=None):
        super().__init__()
        self.metrics = metrics or []
        for metric in self.metrics: metric.register_model(self)
        self._create_model()
        self.pgs = self._get_pgs()
        
    def training_step(self, b, b_idx):
        xb,yb = b
        losses = self(xb,list(yb))
        loss = sum(losses.values())
        log = {'train/loss': loss, **{f'train/{k}':v for k,v in losses.items()}}
        return {'loss': loss, 'log': log}
    
    def validation_step(self, b, b_idx):
        xb,yb = b
        with torch.no_grad():
            self.train(); losses = self(xb,list(yb))
            self.eval();  preds = self(xb)
        loss = sum(losses.values())
        losses = {f'valid/{k}':v for k,v in losses.items()}
        res = {}
        for metric in self.metrics: 
            o = metric.step(xb, yb, preds)
            if notnone(o): raise NotImplementedError # How to update res?
        res.update({'valid/loss': loss, **losses})
        return res
        
    def validation_epoch_end(self, outs):
        res = {}
        for metric in self.metrics:
            o = metric.end(outs)
            if notnone(o): raise NotImplementedError # How to update res?
        log = {k:torch.stack(v).mean() for k,v in mergeds(outs).items()}
        res.update({'val_loss': log['valid/loss'], 'log': log})
        return res
    
    # TODO: Add error "First need to call prepare optimizer"
    def configure_optimizers(self): return [self.opt], [self.sched]
    def prepare_optimizers(self, opt_func, lr, sched_fn):
        self.opt = opt_func(self.get_ps(lr), self.get_lrs(lr)[0])
        self.sched = sched_fn(self.opt)
        
    # TODO: how to be sure all parameters got assigned a lr?
    # already safe because of the check on _rcnn_pgs?
    def get_ps(self, lr):
        lrs = self.get_lrs(lr)
        return [{'params':params(pg), 'lr':lr} for pg,lr in zipsafe(self.pgs,lrs)]
    
    def get_lrs(self, lr): 
        if isinstance(lr, numbers.Number): return [lr]*len(self.pgs)
        elif isinstance(lr, slice): return np.linspace(lr.start, lr.stop, len(self.pgs)).tolist()
        else: raise ValueError(f'lr type {type(lr)} not supported, use a number or a slice')

In [ ]:
#export
def _rcnn_pgs(m):
    body = m.backbone.body
    pgs  = []
    pgs += [nn.Sequential(body.conv1, body.bn1)]
    pgs += [getattr(body,l) for l in list(body) if l.startswith('layer')]
    pgs += [m.backbone.fpn, m.rpn, m.roi_heads]
    if len(params(nn.Sequential(*pgs))) != len(params(m)):
        raise RuntimeError('Malformed model parameters groups, you probably need to use a custom model_splitter')
    return pgs

In [ ]:
#export
class MantisMaskRCNN(RCNN):
    @delegates(MaskRCNN.__init__)
    def __init__(self, n_class, h=256, pretrained=True, metrics=None, **kwargs):
        store_attr(self, 'n_class,h,pretrained,kwargs')
        super().__init__(metrics=metrics)

    def forward(self, images, targets=None): return self.m(images, targets)
        
    def _create_model(self):
        self.m = maskrcnn_resnet50_fpn(pretrained=self.pretrained, **self.kwargs)
        in_features = self.m.roi_heads.box_predictor.cls_score.in_features
        self.m.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.n_class)
        in_features_mask = self.m.roi_heads.mask_predictor.conv5_mask.in_channels
        self.m.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, self.h, self.n_class)
    def _get_pgs(self): return _rcnn_pgs(self.m)

In [ ]:
#export
class MantisFasterRCNN(RCNN):
    @delegates(FasterRCNN.__init__)
    def __init__(self, n_class, h=256, pretrained=True, metrics=None, **kwargs):
        store_attr(self, 'n_class,h,pretrained,kwargs')
        super().__init__(metrics=metrics)
        
    def forward(self, images, targets=None): return self.m(images, targets)
    
    def _create_model(self):
        self.m = fasterrcnn_resnet50_fpn(pretrained=pretrained, **kwargs)
        in_features = self.m.roi_heads.box_predictor.cls_score.in_features
        self.m.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.n_class)
    def _get_pgs(self): return _rcnn_pgs(self.m)

## Predict

In [ ]:
#export
@patch
def predict(self:RCNN, ims=None, rs=None):
    if bool(ims)==bool(rs): raise ValueError('You should either pass ims or rs')
    if notnone(rs): ims = [open_img(o.iinfo.fp) for o in rs]
    xs = [im2tensor(o).to(model_device(self)) for o in ims]
    self.eval()
    return ims, self(xs)

## Visualize

In [ ]:
#export
def show_pred(im, pred, mask_thresh=.5, ax=None):
    # TODO: Implement mask and keypoint
    bboxes,masks,kpts = None,None,None
    if 'boxes' in pred: bboxes = [BBox.from_xyxy(*o) for o in pred['boxes']]
    if 'masks' in pred: masks = Mask(to_np((pred['masks']>.5).long()[:,0,:,:]))
    return show_annot(im, bboxes=bboxes, masks=masks, ax=ax)

In [ ]:
#export
def show_preds(ims, preds, mask_thresh=.5):
    return grid2([partial(show_pred,im=im,pred=pred,mask_thresh=mask_thresh)
                  for im,pred in zip(ims,preds)])

# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00a_core.ipynb.
Converted 00b_lightning_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.core.ipynb.
Converted 04_data.annotations.ipynb.
Converted 06_data.load.ipynb.
Converted 07_transforms.ipynb.
Converted 08_models.ipynb.
Converted 09_learner.ipynb.
Converted 11_metrics.core.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted data_refactor.ipynb.
Converted index.ipynb.
